# ERA vs FIP 分析ノートブック

## 1. 背景・目的
"""
本Notebookでは、プロ野球投手の成績指標である ERA（防御率）と FIP（Fielding Independent Pitching）を用いて、
投手の成績における実力と運の乖離を分析します。
使用データは2024年の規定投球回に達した投手の成績（Excelファイル: bb_an.xlsx）です。
"""

## 2. ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 3. データ読み込みと前処理
# データ読み込み
df = pd.read_excel("bb_an.xlsx")

# 投球回列（例: "144 2/3"）を数値型に変換
def parse_ip(ip_str):
    if isinstance(ip_str, str) and ' ' in ip_str:
        base, frac = ip_str.split(' ')
        if frac == '1/3': return int(base) + 1/3
        elif frac == '2/3': return int(base) + 2/3
        else: return int(base)
    try:
        return float(ip_str)
    except:
        return np.nan

df["投球回"] = df["投球回"].apply(parse_ip)

# 欠損値の削除
df = df.dropna(subset=["投球回", "奪三振", "与四球", "被本塁打", "防御率"])

## 4. FIPの算出
# 補正定数Cを仮に1.0と設定
C = 1.0
df["FIP"] = ((13 * df["被本塁打"] + 3 * df["与四球"] - 2 * df["奪三振"]) / df["投球回"]) + C

# ERAとFIPの乖離
df["ERA-FIP"] = df["防御率"] - df["FIP"]

## 5. 可視化: ERAとFIPの乖離
plt.figure(figsize=(12, 6))
sorted_df = df.sort_values("ERA-FIP", ascending=False)
sns.barplot(x="選手名", y="ERA-FIP", data=sorted_df, palette="coolwarm")
plt.axhline(0, color='black', linestyle='--')
plt.xticks(rotation=90)
plt.title("ERA - FIP 差による運の良し悪し")
plt.xlabel("選手名")
plt.ylabel("ERA - FIP")
plt.tight_layout()
plt.show()

## 6. 考察
"""
- FIPがERAより低い投手は、実力に比べて不運である可能性がある。
- FIPがERAより高い投手は、守備や状況に助けられている可能性がある。
- 全体的にERAとFIPの差は1点未満であるが、明確な傾向を持つ投手も存在。
- 今後、複数年比較や他の指標との相関分析も行うことで、より精緻な評価が可能になる。
"""

## 7. 今後の展望
"""
- クラスタリングによる投手タイプ分類
- 年度別やチーム別の傾向比較
- 回帰分析によるERAの予測
- FIP補正定数の精緻化（リーグ平均ERAからの導出）
"""
